In [2]:
import pandas as pd
import numpy as np

def calccapfacs(df_comp):
    """
    received a df with -> column0:weight ;column1=cap; returns df with additional column2:capfactor; colum3:cappedwgt
    reindexes the df starting with 1
    """

    df_comp = df_comp.sort_values(df_comp.columns[0],ascending=False)
    df_comp.index = range(1,len(df_comp)+1)
    df_comp['capfactor']=1
    if sum(df_comp.iloc[:,1])<=1.:   
        df_comp['cappedwgt'] = 1. / len(df_comp) #equal weight
    else:
        df_comp['cappedwgt'] = df_comp.iloc[:,0]
        while len(df_comp[np.round(df_comp.cappedwgt, 7) > np.round(df_comp.iloc[:,1], 7)]) > 0:
            dblToCap = df_comp[df_comp.cappedwgt >= df_comp.iloc[:,1]].cap.sum()
            weightsnocap = df_comp[df_comp.cappedwgt < df_comp.iloc[:,1]].cappedwgt.sum()
            dblDistFactor = weightsnocap / (1 - dblToCap)
            for index, row in df_comp.iterrows():
                if row['cappedwgt'] >= row[1]: 
                    df_comp.loc[index,'cappedwgt'] = dblDistFactor * row[1]
            dblcappedsum = df_comp.cappedwgt.sum()
            df_comp['cappedwgt'] = df_comp['cappedwgt'] / dblcappedsum
    df_comp['capfactor']=(df_comp['cappedwgt']/df_comp.iloc[:,0])/max(df_comp['cappedwgt']/df_comp.iloc[:,0])
    return df_comp.reset_index(drop=True)


def cap_with_min_devs(dfx, met_=1):
    """
    calculates weights and capping factors minimizing the squared sum of the deviation with the intended weights
    received a df with column0:weight (used as initial guess) ;column1=cap; 
    returns df with additional column2:capfactor; colum3:cappedwgt
    method met_ =1: TNC (Truncated Newton's algorithm); met_ =2: L-BFGS-B (limited memory BFGS)
    reindexes the df starting with 1
    """

    from scipy.optimize import minimize
    dfx['capfactor']=1.
    options={1 : 'TNC',
         2 : 'L-BFGS-B'} #for methods 1 and 2
    def wgtfun(x):
        return sum(x-wt)**2
    x = np.array(dfx.iloc[:,0])
    wt = x
    b = [(0.,dfx.iloc[i,1]) for i in range(len(dfx))]
    c = ({'type':'eq', 'fun': lambda x: sum(x)-1. })  #methods TNC and L-BFGS-B cannot handle constraints
    
    res=minimize(wgtfun, x , method=options[met_], bounds=b, constraints=c)
    dfx['cappedwgt']=res.x
    dfx['capfactor']=(dfx['cappedwgt']/dfx.iloc[:,0])/max(dfx['cappedwgt']/dfx.iloc[:,0])
    return dfx


##dataframe with weights and caps (ordered by weights desc)
df = pd.DataFrame([[0.15,0.15,0.1,0.1,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05],
                   [0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]]).T
df.columns = ['weight','cap']

result1 = calccapfacs(df)
result2 = cap_with_min_devs(df, met_=1)

C:\Users\ROB4473\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\optimize\_minimize.py:433: RuntimeWarning: Method TNC cannot handle constraints.
  RuntimeWarning)


In [7]:
##redistribution of weights proportional
result1

,weight,cap,capfactor,cappedwgt
0,0.15,0.1,0.555556,0.10
1,0.15,0.1,0.555556,0.10
2,0.10,0.1,0.833333,0.10
3,0.10,0.1,0.833333,0.10
4,0.05,0.1,1.000000,0.06
5,0.05,0.1,1.000000,0.06
6,0.05,0.1,1.000000,0.06
7,0.05,0.1,1.000000,0.06
8,0.05,0.1,1.000000,0.06
9,0.05,0.1,1.000000,0.06


In [6]:
##redistribution of weights for minimum weight deviation
result2

,weight,cap,capfactor,cappedwgt
0,0.15,0.1,0.529423,0.100000
1,0.15,0.1,0.529423,0.100000
2,0.10,0.1,0.794134,0.100000
3,0.10,0.1,0.794134,0.100000
4,0.05,0.1,0.944237,0.059451
5,0.05,0.1,0.944237,0.059451
6,0.05,0.1,0.944237,0.059451
7,0.05,0.1,0.944237,0.059451
8,0.05,0.1,1.000000,0.062962
9,0.05,0.1,0.944237,0.059451
